Importing libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Load and understand the Data

In [6]:
df=pd.read_csv(r'C:\Users\ramak\Downloads\Historical Tropical Storm\csv\hyd_v2.csv')
df.head()
df.info()
df.dtypes
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19110 entries, 0 to 19109
Data columns (total 36 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   active              19110 non-null  bool   
 1   amenities           19110 non-null  object 
 2   balconies           19110 non-null  object 
 3   bathroom            19110 non-null  int64  
 4   combineDescription  0 non-null      float64
 5   completeStreetName  19108 non-null  object 
 6   deposit             19110 non-null  int64  
 7   facing              19110 non-null  object 
 8   facingDesc          9569 non-null   object 
 9   floor               19110 non-null  int64  
 10  furnishingDesc      19110 non-null  object 
 11  gym                 19110 non-null  bool   
 12  id                  19110 non-null  object 
 13  isMaintenance       19110 non-null  bool   
 14  lift                19110 non-null  bool   
 15  loanAvailable       19110 non-null  bool   
 16  loca

(19110, 36)

Assumptions and  Data Cleaning

In [33]:
# Standardizing columns names
df.columns=df.columns.str.strip().str.lower().str.replace(" ","_")

# Removing 'combineDescription' and 'weight' as there are no values in it
df.drop(columns=['combinedescription','weight'],inplace=True,errors='ignore')

# In completestreetname i filled Not_Mentioned rather than deleting(2 nans was there).
df['completestreetname'].fillna('Not_Mentioned',inplace=True)

# In facingdesc column i have nan, filled with Unknown rather than removing
df['facingdesc'].fillna('None',inplace=True)

# In locality column i have 6 NaN filled with 'Unknown'
df['locality'].fillna('Unknown',inplace=True)

df['reactivationsource'].fillna('Unknown',inplace=True)

# Converted this column to integer as string data type was there 
df['balconies']=pd.to_numeric(df['balconies'],errors='coerce').fillna(0).astype(int)

# Converted this column to integer as string data type was there
df['maintenanceamount']=pd.to_numeric(df['maintenanceamount'],errors='coerce').fillna(0).astype(int)

Checking for duplicates,empty spaces/strings exist or not

In [45]:
# As i have 969 total duplicate rows,i will remove it
df.duplicated().sum()

# removing duplicates
df=df.drop_duplicates()

# checking for empty spaces/values  (False means everything is okay)
(df=='').any().any()         

# if any empty cells/strings found replace with NaNs 
df.replace(r'^\s*$',pd.NA,regex=True,inplace=True)

In [84]:
# locality column was having inconsistencies like kondapur,kondapur, , ,kondapur, mygate,kondapur
df['locality_clean']=df['locality'].str.split(',').str[-1].str.strip().str.title()
# 
df = df[df['locality_clean'].str.replace(' ', '').str.isalpha()]

Q1. What is the average rent amount by locality?

In [91]:
#df.groupby('locality')['rent_amount'].mean().head()
df.groupby('locality_clean')['rent_amount'].mean().round(0).astype(int).reset_index().rename(columns={'rent_amount':'Avg_Rent_Amount'})

,locality_clean,Avg_Rent_Amount
0,A S Rao Nagar,9338
1,Aarambh Township Flat Owners Association Office,9500
2,Aasritha Residency,7000
3,Abhyudaya Nagar,12000
4,Abids,21833
...,...,...
1471,Zahur Cottage,8500
1472,Zamistanpur,11088
1473,Zamistanpur Post Office,10500
1474,Zehera Nagar,14000


In [93]:
df.groupby('locality_clean')['rent_amount'].mean().round(0).loc['Manikonda']

20761.0

In [94]:
df.head()

,active,amenities,balconies,bathroom,completestreetname,deposit,facing,facingdesc,floor,furnishingdesc,...,property_size,reactivationsource,rent_amount,sharedaccomodation,shorturl,swimmingpool,totalfloor,type_bhk,watersupply,locality_clean
0,True,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",3,3,"Shreya carnation, Block I, NCB Enclave, Gachib...",90000,W,West,3,Semi,...,2200,USER_DASHBOARD,28000,False,http://nobr.kr/sv/9Kvqj,False,5,BHK3,CORP_BORE,Gachibowli
1,True,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",1,2,"Inner Ring Rd, near RTO Bandlaguda South Zone",45000,E,East,2,Unfurnished,...,1200,USER_DASHBOARD,15000,False,http://nobr.kr/sv/j2pD6,False,2,BHK3,BOREWELL,Chandrayangutta
2,True,"{""LIFT"":true,""GYM"":true,""INTERNET"":false,""AC"":...",3,3,"Rd Number 2, Shirdi Sai Nagar, Manikonda, Hyde...",80000,E,East,0,Semi,...,1800,MISSED_CALL,16000,False,http://nobr.kr/sv/TXJmj,False,3,BHK3,CORP_BORE,Manikonda
3,True,"{""LIFT"":false,""GYM"":false,""INTERNET"":false,""AC...",1,2,"Plot No. 44, Road No. 1/A, kakatiya colony, LB...",18000,W,West,2,Unfurnished,...,750,MISSED_CALL,9000,False,http://nobr.kr/sv/FnCsf,False,2,BHK2,CORP_BORE,Lb Nagar
4,True,"{""LIFT"":true,""GYM"":false,""INTERNET"":false,""AC""...",2,2,"Madhapur HUDA Techno Enclave, Near MaxCure Su...",80000,E,East,2,Semi,...,1250,None,32500,False,http://nobr.kr/sv/BQFRb,False,5,BHK2,CORP_BORE,Hitec City
